# Проект Маркетинг

## Описание

Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продажи. Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

### Цель

Предсказать вероятность покупки в течение 90 дней

### Задачи

● Изучить данные  
● Разработать полезные признаки  
● Создать модель для классификации пользователей  
● Улучшить модель и максимизировать метрику roc_auc  
● Выполнить тестирование  

### Данные

apparel-purchases  
история покупок  
● client_id идентификатор пользователя  
● quantity количество товаров в заказе  
● price цена товара  
● category_ids вложенные категории, к которым отнсится товар  
● date дата покупки  
● message_id идентификатор сообщения из рассылки  

apparel-messages  
история рекламных рассылок  
● bulk_campaign_id идентификатор рекламной кампании  
● client_id идентификатор пользователя  
● message_id идентификатор сообщений  
● event тип действия  
● channel канал рассылки  
● date дата рассылки  
● created_at точное время создания сообщения  

apparel-target_binary  
совершит ли клиент покупку в течение следующих 90 дней  
● client_id идентификатор пользователя  
● target целевой признак  


## Подготовка к работе

### Импорты

In [ ]:
# === Стандартная библиотека ===
import time
import re
import ast
import logging
from datetime import datetime
import warnings

# === Научный стек ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# === Визуализация пропусков ===
import missingno as msno

# === IPython / Jupyter ===
from IPython.display import HTML, display
from tqdm import tqdm

# === Статистика ===
from phik import phik_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor

# === Sklearn ===
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    RobustScaler,
    FunctionTransformer,
)
from sklearn.model_selection import (
    train_test_split,
    KFold,
    cross_val_score,
    StratifiedKFold
)
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import roc_auc_score, make_scorer

# === ML-библиотеки ===
from lightgbm import LGBMClassifier
from lightgbm import early_stopping as lgb_early_stopping, log_evaluation as lgb_log_evaluation
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# === Optuna ===
import optuna
from optuna.integration.sklearn import OptunaSearchCV
import optuna.integration.lightgbm as lgb_opt
import optuna.integration.catboost as cat_opt
from optuna.integration import XGBoostPruningCallback, CatBoostPruningCallback, LightGBMPruningCallback
from optuna import Study


# === Настройки ===
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="sklearn.feature_selection._univariate_selection")
logging.getLogger("sklearn").setLevel(logging.ERROR)
optuna.logging.set_verbosity(optuna.logging.WARNING)
pd.set_option("display.max_columns", None)

xgb_params = {"verbosity": 0}

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

### Константы

In [ ]:
RANDOM_STATE = 20
TEST_SIZE = 0.25
N_JOBS = -1
N_ITER = 10000 # число итераций для перебора и поиска лучших параметров
N_CROSS_VALL = 3
EARLY_STOP = 50

### Функции

In [ ]:
# форматирования текста
def format_display(text):
    return HTML(f"<span style='font-size: 1.5em; font-weight: bold; font-style: italic;'>{text}</span>")

# сделаем функцию оценки пропусков в датасетах
def missing_data(data):
    missing_data = data.isna().sum()
    missing_data = missing_data[missing_data > 0]
    display(missing_data)

# функция для обработки пробелов
def process_spaces(s):
    if isinstance(s, str):
        s = s.strip()
        s = ' '.join(s.split())
    return s

# замена пробелов на нижнее подчеркинвание в названии столбцов
def replace_spaces(s):
    if isinstance(s, str):
        s = s.strip()
        s = '_'.join(s.split())
    return s

def drop_duplicated(data):
    # проверка дубликатов
    display(format_display("Проверим дубликаты и удалим, если есть"))
    num_duplicates = data.duplicated().sum()
    display(num_duplicates)
    
    if num_duplicates > 0:
        display("Удаляем")
        data = data.drop_duplicates(keep='first').reset_index(drop=True)  # обновляем DataFrame
    else:
        display("Дубликаты отсутствуют")
    return data

def normalize_columns(columns):
    new_cols = []
    for col in columns:
        # вставляем "_" перед заглавной буквой (латиница или кириллица), кроме первой
        col = re.sub(r'(?<!^)(?=[A-ZА-ЯЁ])', '_', col)
        # приводим к нижнему регистру
        col = col.lower()
        new_cols.append(col)
    return new_cols

def check_data(data):
    # приведем все к нижнему регистру
    data.columns = normalize_columns(data.columns)
    
    # удалим лишние пробелы в строках
    data = data.map(process_spaces)

    # и в названии столбцов
    data.columns = [replace_spaces(col) for col in data.columns]
    
    # общая информация 
    display(format_display("Общая информация базы данных"))
    display(data.info())
    
    # 5 строк
    display(format_display("5 случайных строк"))
    display(data.sample(5))
    
    # пропуски
    display(format_display("Число пропусков в базе данных"))
    display(missing_data(data))

    # проверка на наличие пропусков
    if data.isnull().sum().sum() > 0:
        display(format_display("Визуализация пропусков"))
        msno.bar(data)
        plt.show()
        
    # средние характеристики
    display(format_display("Характеристики базы данных"))
    display(data.describe().T)
    
    # data = drop_duplicated(data)
    
    return data  # возвращаем измененные данные

def parse_category_ids(x):
    if isinstance(x, str):
        return ast.literal_eval(x)
    return x

def plot_combined(data, col=None, target=None, col_type=None, legend_loc='best'):
    """
    Строит графики для числовых столбцов в DataFrame, автоматически определяя их типы (дискретные или непрерывные).

    :param data: DataFrame, содержащий данные для визуализации.
    :param col: Список столбцов для построения графиков. Если None, будут использованы все числовые столбцы.
    :param target: Столбец, по которому будет производиться разделение (для hue в графиках).
    :param col_type: Словарь, определяющий типы столбцов ('col' для непрерывных и 'dis' для дискретных).
                     Если None, типы будут определены автоматически.
    :param legend_loc: Положение легенды для графиков (по умолчанию 'best').
    :return: None. Графики отображаются с помощью plt.show().
    """
    
    # Определяем числовые столбцы
    if col is None:
        numerical_columns = data.select_dtypes(include=['int', 'float']).columns.tolist()
    else:
        numerical_columns = col

    # Если col_type не указан, определяем типы автоматически
    if col_type is None:
        col_type = {}
        for col in numerical_columns:
            unique_count = data[col].nunique()
            if unique_count > 20:
                col_type[col] = 'col'  # Непрерывные данные
            else:
                col_type[col] = 'dis'  # Дискретные данные

    total_plots = len(numerical_columns) * 2
    ncols = 2
    nrows = (total_plots + ncols - 1) // ncols

    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 5 * nrows))
    axs = axs.flatten()

    index = 0

    for col in numerical_columns:
        # Определяем тип графика
        plot_type = col_type.get(col)
        if plot_type is None:
            raise ValueError(f"Тип для столбца '{col}' не указан в col_type.")

        # Гистограмма или countplot
        if index < len(axs):
            if plot_type == 'col':
                if target is not None:
                    sns.histplot(data, x=col, hue=target, bins=20, kde=True, ax=axs[index])
                    handles, labels = axs[index].get_legend_handles_labels()
                    if handles:
                        axs[index].legend(title=target, loc=legend_loc)
                else:
                    sns.histplot(data[col].dropna(), bins=20, kde=True, ax=axs[index])
                axs[index].set_title(f'Гистограмма: {col}')
            elif plot_type == 'dis':
                if target is not None:
                    sns.countplot(data=data, x=col, hue=target, ax=axs[index])
                    handles, labels = axs[index].get_legend_handles_labels()
                    if handles:
                        axs[index].legend(title=target, loc=legend_loc)
                else:
                    sns.countplot(data=data, x=col, ax=axs[index])
                axs[index].set_title(f'Countplot: {col}')
                # поворот подписей X для дискретных
                axs[index].tick_params(axis='x', rotation=90)
            index += 1

        # Боксплот
        if index < len(axs):
            sns.boxplot(x=data[col], ax=axs[index])
            axs[index].set_title(f'Боксплот: {col}')
            # тоже поворачиваем, если дискретные значения
            if plot_type == 'dis':
                axs[index].tick_params(axis='x', rotation=90)
            index += 1

    # Отключаем оставшиеся оси
    for j in range(index, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()
    
def calc_target_correlations(df, target_col: str = None, drop_cols: list = None): # type: ignore
    """
    Считает корреляции признаков с таргетом, строит heatmap и рассчитывает VIF.
    Результаты выводятся прямо в Jupyter.
    """
    if drop_cols is None:
        drop_cols = []
    
    df_tmp = df.copy()

    # Преобразуем категориальные в числовые
    cat_cols = df_tmp.select_dtypes(include=["object", "category"]).columns
    for c in cat_cols:
        df_tmp[c] = df_tmp[c].astype("category").cat.codes

    # Числовые колонки
    numeric_cols = df_tmp.select_dtypes(exclude=["object", "category"]).columns.tolist()
    if target_col not in numeric_cols:
        raise ValueError(f"target_col '{target_col}' должен быть числовым")

    # Корреляции с target
    corr_df = (
        df_tmp[numeric_cols]
        .corr()[target_col]
        .drop(target_col)
        .sort_values(key=np.abs, ascending=False)
    )
    display("=== Корреляция с таргетом ===")
    display(corr_df)

    # Heatmap
    heatmap_cols = [col for col in numeric_cols if col not in drop_cols or col == target_col]
    corr_matrix = df_tmp[heatmap_cols].corr()

    plt.figure(figsize=(12, 10))
    plt.imshow(corr_matrix, interpolation="nearest", cmap="coolwarm", aspect="auto")
    plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90, fontsize=8)
    plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns, fontsize=8)
    plt.colorbar()
    plt.title("Correlation Heatmap (включая target)")

    for i in range(corr_matrix.shape[0]):
        for j in range(corr_matrix.shape[1]):
            value = corr_matrix.iloc[i, j]
            plt.text(j, i, f"{value:.2f}", ha="center", va="center", fontsize=5, color="black")

    plt.tight_layout()
    plt.show()

    # VIF
    vif_cols = [col for col in numeric_cols if col != target_col and col not in drop_cols]
    X_vif = df_tmp[vif_cols].copy()
    scaler = RobustScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_vif), columns=vif_cols)

    vif_data = pd.DataFrame()
    vif_data["feature"] = vif_cols
    vif_data["VIF"] = [
        variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])
    ]
    vif_data = vif_data.sort_values("VIF", ascending=False)

    display("=== VIF ===")
    display(vif_data)

In [ ]:
class EarlyStoppingCallback:
    def __init__(self, patience=EARLY_STOP, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_score = -np.inf
        self.no_improvement_count = 0

    def __call__(self, study, trial):
        if study.best_value > self.best_score + self.min_delta:
            self.best_score = study.best_value
            self.no_improvement_count = 0
        else:
            self.no_improvement_count += 1

        if self.no_improvement_count >= self.patience:
            study.stop()
            logger.info(f"Ранняя остановка: нет улучшений {self.patience} trials")

## EDA

### Подключим и почитаем данные

In [ ]:
app_msg = pd.read_csv("../data/apparel-messages.csv")
app_prch = pd.read_csv("../data/apparel-purchases.csv")
app_target = pd.read_csv("../data/apparel-target_binary.csv")
event_type = pd.read_csv("../data/full_campaign_daily_event.csv")
event_chanel = pd.read_csv("../data/full_campaign_daily_event_channel.csv")

### Первичная оценка и обработка данных

#### app_msg

In [ ]:
app_msg = check_data(app_msg)

In [ ]:
# посмотрим дату начала и конца событий
first_date = app_msg['date'].min()
last_date = app_msg['date'].max()
display(first_date)
display(last_date)

In [ ]:
# посмотрим уники среди событий и канал распространения
display(app_msg['event'].unique())
display(app_msg['channel'].unique())

Что имеем:  
open — письмо открыто  
click — клик по ссылке в письме  
purchase — покупка после перехода из письма  
send — отправка письма  
unsubscribe — отписка от рассылки  
hbq_spam — сообщение отмечено как спам  
hard_bounce — письмо не доставлено из-за постоянной ошибки (адрес не существует)  
subscribe — подписка на рассылку  
soft_bounce — письмо не доставлено из-за временной ошибки (ящик переполнен, сервер недоступен)  
complain — жалоба пользователя (напр. “Это спам”)  
close — завершение сессии (иногда: закрытие письма или вкладки)  

In [ ]:
# посмотри число дубликатов в базе
app_msg.duplicated().sum()

Вот тут тонкий момент:  
1) Это косяк базы и надо просто удалить дубликаты и забыть про них;  
2) Сомнения имею я, что это косяк и скорее всего это спамер просто отправил несколько мессаг и тогда из этого можно сделать бинарную фичу, что-то типа "spam_factor"

In [ ]:
# проверим сколько уникальных клиентов
display(app_msg['client_id'].nunique())

Из 12.739.798 строк мы имеем 53.329 уникальных клиентов.  
Нужна будет пересборка данных с агрегацией

Выводы:  
Самая объемная база.  
Столбцы  date, created_at имеют неверный формат данных - необходимо будет преобразовать.


#### app_prch

In [ ]:
app_prch = check_data(app_prch)

In [ ]:
# проверим сколько уникальных клиентов совершило покупки
display(app_prch['client_id'].nunique())

In [ ]:
display(app_prch['client_id'].nunique() / app_msg['client_id'].nunique())

Т.е. после всех событий произвели покупку 93.4% уникальных пользователей в текущей выборке, и почти 7% проигнорировало.  
В целом это очень хороший показатель.

In [ ]:
# и еще посмотрим категории
app_prch[app_prch['category_ids'] == '[]']

In [ ]:
# тут еще заодно проведем более глубокий анализ данных
plot_combined(app_prch, col=None, target=None, col_type=None, legend_loc='best')

In [ ]:
# здесь нам интересно глянуть цены, сколько нулей и где
app_prch[app_prch['price'] < 3]

Нулевых нет, уже хорошо.  
Ну, а цена в одну единицу в целом бывает.  
Также видим, что появились непонятные категории - пустые или None, что тоже не очень хорошо, но их обработает дальше.

Выводы:  
Столбец  date имеет неверный формат данных - необходимо будет преобразовать.  
Столбец category_ids - тут 2 варианта развития событий:  
- возьмем только глобальную категорию и конкретный товар, т.е. 1е и последнее значение;  
- можно будет преобразовывать в разряженную матрица, т.к. небольшой объем данных  
ну и потом с такой матрицей умеет работать xgboost который и будем использовать в предсказаниях;

#### app_target

In [ ]:
app_target = check_data(app_target)

Выводы:  
Ну тут все понятно, обсуждать нечего

#### event_type

In [ ]:
event_type = check_data(event_type)

Вывод:  
Здесь просто агрегированная статистика по событиям, не вижу смысла, что-то делать с этой таблицей в принципе

#### event_chanel

In [ ]:
event_chanel = check_data(event_chanel)

Вывод:  
По сути своей, тоже самое, что и прошлая таблица - статистика по событиям.  
Возможно и есть смысл где-то использовать, но пока непонятно где и как

#### Выводы

Были подгружены и изучены предоставленные данные.  
Глобально, для реализации задачи нам понадобится только 3 таблицы - apparel-messages, apparel-purchases и apparel-target_binary, т.к. эти таблицы несут основную смысловую нагрузку.  
Две оставшиеся таблицы - статистика по ивентам и активности пользователей без привязки к этим самым пользователям и ничего нам не дадут.  



## Предобработка данных

### Обработка имеющихся данных

In [ ]:
# преобразуем даты
app_msg["date"] = pd.to_datetime(app_msg["date"], errors="coerce")
app_msg["created_at"] = pd.to_datetime(app_msg["created_at"], errors="coerce")

app_prch["date"] = pd.to_datetime(app_prch["date"], errors="coerce")

### feature engineering

Для начала поработаем с каждой таблицей по отдельности и сделаем, что-то новое.  

app_msg
Что мы можем сделать:  
1) У нас есть дата реакции на ивент и дата создания ивента - скорость реакции на ивент, потом усредняем;  
2) Есть канал и действие - соберем суммарное число действий по каждому каналу;  
3) Сделать бинарную фичу "spam_factor", т.е. берем дубли, смотрим кого заспамили - тем 1, кого нет - 0.  

app_prch  
Что мы можем сделать:  
1) Время с момента последней покупки, сделаем в днях;  
2) Среднее число товаров в заказе;  
3) Средний чек заказа;  
4) Средняя цена итема;  
5) В какой категории больше всего покупок (предварительно разобьем category_ids на top и last id - глобальную категорию и конкретный товар);  
6) Любимый товар;  
7) Любимая категория;  
8) Средний интервал между покупками, в днях;  
9) Дней с последнего взаимодействия с рассылкой;  
10) Сделаем группировку по 30/60/90/180/360 дней, а там агрегируем по числу покупок, числу итемов, сумме затрат от последней имеющейся у нас отчетной даты;  

#### app_msg

In [ ]:
app_msg.head()

In [ ]:
# У нас есть дата реакции на ивент и дата создания ивента - скорость реакции
app_msg["reaction_for_event"] = (
    (app_msg["date"].dt.normalize() - app_msg["created_at"].dt.normalize())
    .dt.days.astype("int16")
)

In [ ]:
app_msg['reaction_for_event'].unique()

Неожиданно, все реагируют на ивенты в тот же день.  
В таком случае этот признак для нас бесполезен

In [ ]:
# теперь агрегируем данные
app_msg_agg = (
    app_msg.groupby("client_id")
    .agg(
        bulk_campaigns=("bulk_campaign_id", "nunique"),
        messages=("message_id", "nunique"),
        events=("event", "nunique"),
        channels=("channel", "nunique"),
        first_date=("date", "min"),
        last_date=("date", "max"),
        pop_event=("event", lambda x: x.value_counts().idxmax() if not x.value_counts().empty else "unknown")
    )
    .reset_index()
)

In [ ]:
# считаем число действий по каналам и событиям
channel_event_counts = (
    pd.crosstab(
        index=app_msg["client_id"],
        columns=[app_msg["channel"], app_msg["event"]]
    )
)

# делаем имена колонок: actions_{channel}_{event}
channel_event_counts.columns = [
    f"actions_{ch}_{ev}" for ch, ev in channel_event_counts.columns
]

# собираем в кучу
channel_event_counts = channel_event_counts.reset_index()
app_msg_agg = app_msg_agg.merge(channel_event_counts, on="client_id", how="left")

In [ ]:
# считаем количество дублей сообщений на юзера
dup_counts = (
    app_msg.groupby("client_id")
    .agg(dup_count=("message_id", lambda x: app_msg.loc[x.index].duplicated().sum()))
    .reset_index()
)

# spam_factor = 1 если дублей >= 2, иначе 0
dup_counts["spam_factor"] = (dup_counts["dup_count"] >= 2).astype(int)

# объединяем с агрегатами
app_msg_agg = app_msg_agg.merge(dup_counts, on="client_id", how="left")
app_msg_agg[["dup_count", "spam_factor"]] = app_msg_agg[["dup_count", "spam_factor"]].fillna(0).astype(int)

In [ ]:
app_msg_agg[app_msg_agg["spam_factor"] == 1]

In [ ]:
display(app_msg_agg.head())
display(app_msg_agg.info())

Выглядит жутко, но мы закрываем глаза и продолжаем делать :)

#### app_prch

In [ ]:
# начнем с разбора category_ids
# превращаем в списки category_ids
app_prch["category_ids"] = app_prch["category_ids"].apply(parse_category_ids)

# берём первую категорию
app_prch["category_ids_top"] = app_prch["category_ids"].apply(
    lambda x: int(x[0]) if len(x) > 0 and str(x[0]).isdigit() else -1
).astype("int16")

# берём последнюю непустую категорию
app_prch["category_ids_last"] = app_prch["category_ids"].apply(
    lambda x: int(next((i for i in reversed(x) if i not in [None, "", "nan"] and str(i).isdigit()), -1))
).astype("int16")

In [ ]:
# время с момента последней покупки (в днях)
last_date = pd.to_datetime(last_date)

last_purchase = (
    app_prch.groupby("client_id")["date"].max()
    .reset_index(name="last_purchase_date")
)

last_purchase["days_from_last_purchase"] = (
    (last_date - last_purchase["last_purchase_date"]).dt.days.astype("int16")
)
last_purchase.head()

In [ ]:
# среднее число товаров в заказе
avg_items_per_order = (
    app_prch.groupby(["client_id", "message_id"])["quantity"].sum()
    .groupby("client_id").median()
    .reset_index(name="avg_items_per_order")
)
avg_items_per_order.head()

In [ ]:
# средний чек заказа
avg_order_sum = (
    app_prch.groupby(["client_id", "message_id"])
    .apply(lambda df: (df["quantity"] * df["price"]).sum(), include_groups=False)
    .groupby("client_id").mean()
    .reset_index(name="avg_order_sum")
)
avg_order_sum.head()

In [ ]:
# средняя цена итема
avg_item_price = (
    (app_prch["price"] * app_prch["quantity"])
    .groupby(app_prch["client_id"]).sum()
    / app_prch.groupby("client_id")["quantity"].sum()
)
avg_item_price = avg_item_price.reset_index(name="avg_item_price")
avg_item_price.head()

In [ ]:
# любимая категория (top)
fav_category_top = (
    app_prch.groupby("client_id")["category_ids_top"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "unknown")
    .reset_index(name="fav_category_top")
)
fav_category_top.head()

In [ ]:
# любимый товар (last)
fav_category_last = (
    app_prch.groupby("client_id")["category_ids_last"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "unknown")
    .reset_index(name="fav_category_last")
)
fav_category_last.head()

In [ ]:
# средний интервал между покупками (в днях)
avg_interval = (
    app_prch.groupby("client_id")["date"]
    .apply(
        lambda x: np.mean(np.diff(np.sort(x.unique())).astype("timedelta64[D]").astype(int))
        if x.nunique() > 1 else -1
    )
    .reset_index(name="avg_interval_days")
)
display(avg_interval.sample(5))
display(avg_interval.describe().T)

In [ ]:
# сумма покупок
lifetime_value = (
    (app_prch["quantity"] * app_prch["price"])
    .groupby(app_prch["client_id"]).sum()
    .reset_index(name="lifetime_value")
)
display(lifetime_value.head())
display(lifetime_value.describe().T)

In [ ]:
# churn-флаг (>360 дней без покупок)
churn_flag = (
    app_prch.groupby("client_id")["date"].max()
    .reset_index(name="last_purchase_date")
)
churn_flag["churn_flag"] = (
    (last_date - churn_flag["last_purchase_date"]).dt.days > 360
).astype("int8")
churn_flag.head()

In [ ]:
# агрегации по окнам 30/60/90/180/360 дней
def agg_period(df, cutoff_days, all_clients):
    cutoff = last_date - pd.Timedelta(days=cutoff_days)
    dff = df[df["date"] >= cutoff]
    out = dff.groupby("client_id").agg(
        purchases=("message_id", "nunique"),
        items=("quantity", "sum"),
        cost=("price", lambda x: (dff.loc[x.index, "quantity"] * x).sum())
    )

    out = out.rename(columns={
        "purchases": f"purchases_{cutoff_days}d",
        "items": f"items_{cutoff_days}d",
        "cost": f"cost_{cutoff_days}d"
    })

    # добавляем всех клиентов, которых нет в dff
    out = out.reindex(all_clients, fill_value=0).reset_index()
    return out

all_clients = app_prch["client_id"].unique()
agg_30 = agg_period(app_prch, 30, all_clients)
agg_60 = agg_period(app_prch, 60, all_clients)
agg_90 = agg_period(app_prch, 90, all_clients)
agg_180 = agg_period(app_prch, 180, all_clients)
agg_360 = agg_period(app_prch, 360, all_clients)

agg_360.head()

In [ ]:
# объединяем все в единый датафрейм client_features
app_prch_agg = last_purchase[["client_id", "days_from_last_purchase"]] \
    .merge(avg_items_per_order, on="client_id", how="left") \
    .merge(avg_order_sum, on="client_id", how="left") \
    .merge(avg_item_price, on="client_id", how="left") \
    .merge(fav_category_top, on="client_id", how="left") \
    .merge(fav_category_last, on="client_id", how="left") \
    .merge(avg_interval, on="client_id", how="left") \
    .merge(lifetime_value, on="client_id", how="left") \
    .merge(churn_flag[["client_id", "churn_flag"]], on="client_id", how="left") \
    .merge(agg_30, on="client_id", how="left") \
    .merge(agg_60, on="client_id", how="left") \
    .merge(agg_90, on="client_id", how="left") \
    .merge(agg_180, on="client_id", how="left") \
    .merge(agg_360, on="client_id", how="left")

app_prch_agg.head()

In [ ]:
app_prch_agg['client_id'].nunique()

In [ ]:
app_msg_agg['client_id'].nunique()

### Финальная таблица

In [ ]:
# ну и финально смержим 2 агрегированные таблицы и таблицу с таргетом
df = app_prch_agg.merge(app_msg_agg, on='client_id', how='left')
df = df.merge(app_target, on='client_id', how='left')

In [ ]:
display(df.head())

In [ ]:
display(df.describe().T)

In [ ]:
# оценим пропуски
missing = df.isnull().sum()
missing = missing[missing > 0]
display(missing)

In [ ]:
# как оказалось списки клиентов не бьются в файла и есть неучтенные
# столбцы для заполнения "unknown"
cols_unknown = ["first_date", "last_date", "pop_event"]

# все остальные перечисленные числовые столбцы
cols_numeric = [
    "bulk_campaigns", "messages", "events", "channels",
    "actions_email_click", "actions_email_complain", "actions_email_hard_bounce",
    "actions_email_hbq_spam", "actions_email_open", "actions_email_purchase",
    "actions_email_send", "actions_email_soft_bounce", "actions_email_subscribe",
    "actions_email_unsubscribe", "actions_mobile_push_click", "actions_mobile_push_close",
    "actions_mobile_push_hard_bounce", "actions_mobile_push_open", "actions_mobile_push_purchase",
    "actions_mobile_push_send", "actions_mobile_push_soft_bounce",
    "dup_count", "spam_factor"
]

# заменяем пропуски
df[cols_unknown] = df[cols_unknown].fillna("unknown")
df[cols_numeric] = df[cols_numeric].fillna(0)

In [ ]:
for col, dtype in zip(df.columns, df.dtypes):
    display(f"{col}: {dtype}")

### Корелляция

In [ ]:
# таблица корелляции получилась дикой, поэтому сразу буду прописывать в дропы после last_date фичи которые коррелируют в единицу с чем-либо без дополнительных выводов + vif сюда же
# с точки зрения бизнеса я конечно же не прав, но и захламлять не вижу смысла файл
drop_cols = ['client_id', 'first_date', 'last_date', 'messages', 'bulk_campaigns', 'cost_90d', 'items_90d', 'churn_flag']

In [ ]:
df_for_train = df.drop(columns=drop_cols)
calc_target_correlations(df_for_train, target_col="target")

vif у этих фич получился не очень... будем убирать по одному (выше уже будет убранный вариант)  
25	messages	4751.468827045559  
24	bulk_campaigns	4557.404604056862  
44	actions_mobile_push_send	86.63033251646662  
35	actions_email_send	37.31511841292837  
17	cost_90d	32.38699214915867  
14	cost_60d	27.655784671292132  
8	churn_flag	23.014173540405533  
16	items_90d	21.396144750100234  
13	items_60d	18.35091376893147  
15	purchases_90d	17.067579757645326  
12	purchases_60d	14.87017587054433  
21	purchases_360d	13.347259333650094  
20	cost_180d	10.067449835369827  

## Обучение модели

### Подготовка

In [ ]:
# возьмем часть данных для подбора гиперпараметров и финального теста
df_test = df_for_train.sample(frac=0.05, random_state=RANDOM_STATE)
df_rest = df_for_train.drop(df_test.index)

In [ ]:
# разделим на выборки из оставшихся данных
X_train, X_val, y_train, y_val = train_test_split(
    df_rest.drop(['target'], axis=1),
    df_rest['target'],
    test_size=TEST_SIZE,
    stratify=df_rest['target'],
    random_state=RANDOM_STATE
)

# финальный отложенный тест
X_final_test = df_test.drop(['target'], axis=1)
y_final_test = df_test['target']

In [ ]:
# сюда будем писать результаты
results = {}

In [ ]:
# Разделяем признаки
cat_selector = make_column_selector(dtype_include=["object", "category"])
num_selector = make_column_selector(dtype_exclude=["object", "category"])

In [ ]:
cat_cols = cat_selector(X_train)
num_cols = num_selector(X_train)

display("Категориальные:", cat_cols)
display("Числовые:", num_cols)

In [ ]:
to_str = FunctionTransformer(lambda x: x.astype(str))

# для линейных моделей
preprocessor_linear = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("scaler", StandardScaler())
        ]), num_selector),
        ("cat", Pipeline([
            ("to_str", to_str),
            ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False))
        ]), cat_selector)
    ]
)

# для деревьев и бустингов
preprocessor_tree = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_selector),
        ("cat", Pipeline([
            ("to_str", to_str),
            ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
        ]), cat_selector)
    ]
)

In [ ]:
models = {
    "LogisticRegression": (lambda: LogisticRegression(random_state=RANDOM_STATE), preprocessor_linear),
    "RandomForest": (lambda: RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=N_JOBS), preprocessor_tree),
    "DecisionTree": (lambda: DecisionTreeClassifier(random_state=RANDOM_STATE), preprocessor_tree),
    "LightGBM": (lambda: LGBMClassifier(random_state=RANDOM_STATE, n_jobs=N_JOBS, verbosity=-1), preprocessor_tree),
    "XGBoost": (lambda: XGBClassifier(random_state=RANDOM_STATE, n_jobs=N_JOBS, verbosity=0, use_label_encoder=False), preprocessor_tree),
    "CatBoost": (lambda: CatBoostClassifier(random_state=RANDOM_STATE, task_type="CPU", thread_count=N_JOBS, verbose=0), preprocessor_tree),
}

In [ ]:
def objective_model(trial, model_name, preprocessor):
    # разделяем train/val внутри trial
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train,
        test_size=0.25,
        stratify=y_train,
        random_state=RANDOM_STATE
    )

    # число признаков для SelectKBest
    k_best = trial.suggest_int("selectkbest__k", 5, X_train.shape[1])

    # настраиваем гиперпараметры конкретной модели
    if model_name == "LogisticRegression":
        C = trial.suggest_float("C", 0.01, 10, log=True)
        model = LogisticRegression(C=C, random_state=RANDOM_STATE, max_iter=1000)
    elif model_name == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
        max_depth = trial.suggest_int("max_depth", 3, 20)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                       random_state=RANDOM_STATE, n_jobs=N_JOBS)
    elif model_name == "DecisionTree":
        max_depth = trial.suggest_int("max_depth", 3, 20)
        model = DecisionTreeClassifier(max_depth=max_depth, random_state=RANDOM_STATE)
    elif model_name == "LightGBM":
        n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
        max_depth = trial.suggest_int("max_depth", 3, 20)
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
        model = LGBMClassifier(
            n_estimators=n_estimators, max_depth=max_depth,
            learning_rate=learning_rate, random_state=RANDOM_STATE,
            n_jobs=N_JOBS, verbosity=-1
        )
    elif model_name == "XGBoost":
        n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
        max_depth = trial.suggest_int("max_depth", 3, 20)
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
        model = XGBClassifier(
            n_estimators=n_estimators, max_depth=max_depth,
            learning_rate=learning_rate, random_state=RANDOM_STATE,
            n_jobs=N_JOBS, verbosity=0, use_label_encoder=False
        )
    elif model_name == "CatBoost":
        iterations = trial.suggest_int("iterations", 100, 1000, step=50)
        depth = trial.suggest_int("depth", 3, 10)
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
        model = CatBoostClassifier(
            iterations=iterations, depth=depth,
            learning_rate=learning_rate, random_state=RANDOM_STATE,
            task_type="CPU", thread_count=N_JOBS, verbose=0
        )

    # pipeline с SelectKBest
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("selectkbest", SelectKBest(score_func=f_classif, k=k_best)),
        ("classifier", model)
    ])

    # обучаем
    if model_name in ["LightGBM", "XGBoost", "CatBoost"]:
        # Для бустингов — используем eval_set и report для pruner
        pipeline.fit(X_tr, y_tr)
        y_pred = pipeline.predict_proba(X_val)[:, 1]
    else:
        pipeline.fit(X_tr, y_tr)
        y_pred = pipeline.predict_proba(X_val)[:, 1]

    auc = roc_auc_score(y_val, y_pred)

    # пробуем передать в Optuna значение для прунинга
    trial.report(auc, step=0)
    if trial.should_prune():
        raise optuna.TrialPruned()

    return auc

In [ ]:
best_models = {}
study_results = []

early_stop_cb = EarlyStoppingCallback(patience=EARLY_STOP)

def log_every_N_trials(study, trial):
    if trial.number % 100 == 0:
        logger.info(f"[Trial {trial.number}] ROC_AUC={trial.value:.4f}, params={trial.params}")

for model_name, (model_factory, preprocessor) in models.items():
    logger.info(f"\n\n=== Optimizing {model_name} ===")
    
    # pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0)
    study = optuna.create_study(direction="maximize", pruner=pruner)

    study.optimize(
        lambda trial: objective_model(trial, model_name, preprocessor),
        n_trials=N_ITER,
        callbacks=[log_every_N_trials, early_stop_cb]
    )

    best_params = study.best_params
    best_value = study.best_value

    logger.info(f"Best trial -> ROC_AUC={best_value:.4f}, params={best_params}")

    k_best_final = best_params.get("selectkbest__k", X_train.shape[1])
    final_pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("selectkbest", SelectKBest(score_func=f_classif, k=k_best_final)),
        ("classifier", model_factory())
    ])

    model_params = {k.replace("classifier__", ""): v for k, v in best_params.items() if k.startswith("classifier__")}
    if model_params:
        final_pipe.named_steps["classifier"].set_params(**model_params)

    final_pipe.fit(X_train, y_train)
    best_models[model_name] = final_pipe

    study_results.append({
        "Model": model_name,
        "Best_params": best_params,
        "ROC_AUC_CV": best_value
    })

### Подбор лучшей модели

### Сравнение результатов

In [ ]:
# Добавим колонки для ROC_AUC на X_val и X_final_test
for res in study_results:
    name = res["Model"]
    model = best_models[name]

    # Предсказания вероятностей для положительного класса
    y_val_pred = model.predict_proba(X_val)[:, 1]
    y_test_pred = model.predict_proba(X_final_test)[:, 1]

    # Вычисляем ROC AUC
    res["ROC_AUC_Val"] = roc_auc_score(y_val, y_val_pred)
    res["ROC_AUC_FinalTest"] = roc_auc_score(y_final_test, y_test_pred)

# Пересоздаём DataFrame с обновлёнными результатами
results_df = pd.DataFrame(study_results)
display(results_df.sort_values("ROC_AUC_FinalTest", ascending=False))